In [ ]:
import os
import sys
import fnmatch
import numpy as np
import pandas as pd
import json
import gzip
import pickle
import csv
import scipy.sparse
Xauth = None
from collections import defaultdict

In [ ]:
# setup the update to work despite the broken scipy documentation
try:
    a = scipy.sparse.dok_matrix((10,10))
    a.update({(0,0):1.0})
    scipy.sparse.dok_matrix.my_update = scipy.sparse.dok_matrix.update
except:
    a = scipy.sparse.dok_matrix((10,10))
    a._update({(0,0):1.0})
    scipy.sparse.dok_matrix.my_update = scipy.sparse.dok_matrix._update

In [ ]:
with gzip.open('useful_venue_list.pkl.gz','rb') as fp:
    all_venues = pickle.load(fp)
with gzip.open('useful_authors_list.pkl.gz','rb') as fp:
    all_authors = pickle.load(fp)
with gzip.open('useful_papers.pkl.gz','rb') as fp:
    all_papers = pickle.load(fp)

In [ ]:
conf_idx = {v:i for i,v in enumerate(all_venues)}
name_idx = {v:i for i,v in enumerate(all_authors)}
n_confs = len(all_venues)
n_auths = len(all_authors)
print(n_confs,n_auths)

In [ ]:
faculty_affil = pd.read_csv('faculty-affiliations.csv')
ranks = pd.read_csv('other_ranks/ranks.csv')
def csv2dict_str_str(fname):
    with open(fname, mode='r') as infile:
        rdr = csv.reader(infile)
        d = {rows[0].strip(): rows[1].strip() for rows in rdr}
    return d
aliasdict = csv2dict_str_str('dblp-aliases-expanded.csv')

In [ ]:
min_year = all_papers[0][6]
max_year = all_papers[-1][6]
span_years = max_year - min_year + 1
print(min_year,max_year,span_years)

In [ ]:
curious_names = [
    'Aditya Dhawale',
    'Tesca Fitzgerald',
    'Adam W. Harley',
    "Xiaolong Wang 0004",
    "Judy Hoffman",
    "Paris Siminelakis",
    "Roie Levin",
    "Leonid Keselman",
    "Rick Goldstein",
    "Nicholas Rhinehart",
    "Vincent Sitzmann",
    "Siddharth Ancha",
    "Xingyu Lin",
    "Humphrey Hu",
    "David F. Fouhey",
    "Chelsea Finn",
    "Dinesh Jayaraman",
    "Wen Sun 0002",
    "Lerrel Pinto",
    "Justin Johnson 0001",
    "Amir Zamir",
    "Dominik Peters",
    "Jonathan T. Barron",
    "Dorsa Sadigh",
    "Derek Hoiem",
    "Vaggos Chatziafratis",
    "Brian Okorn",
    "David Held"
]

curious_names2 = ['Pulkit Agrawal',
 'Joydeep Biswas',
 'Katherine L. Bouman',
 'David Braun',
 'Jia Deng',
 'Naomi T. Fitter',
 'David F. Fouhey',
 'Saurabh Gupta',
 'Judy Hoffman',
 'Hanbyul Joo',
 'Honglak Lee',
 'Changliu Liu',
 'Petter Nilsson',
 "Matthew O'Toole",
 'Alessandro Roncone',
 'Alanson P. Sample',
 'Manolis Savva',
 'Adriana Schulz',
 'Amy Tabb',
 'Fatma Zeynep Temel',
 'Long Wang',
 'Ling-Qi Yan']

In [ ]:
ri_names = list(set([aliasdict.get(row[1],row[1]) for row in pd.read_csv('other_ranks/cmu_faculty.csv').itertuples() if row[2] == 'RI']))

In [ ]:
scoreV = None
try:
    import gzip
    import pickle
    with gzip.open('scoresV2.pkl.gz','rb') as fp:
        scoreV = pickle.load(fp)
except:
    print('failed!')

In [ ]:
conf_types = {
        'RI': [_[1] for _ in sorted(ri_fav_confs,reverse=True) if _[-2] >= 1.25],
        'ML':['NIPS','ICML','AAAI','AISTATS','IJCAI','UAI','CoRL','ICLR'],
        'CV':['CVPR','ICCV','ECCV','IEEE Trans. Pattern Anal. Mach. Intell.','FGR','Int. J. Comput. Vis.','WACV','BMVC'],
        'ROB':['HRI','Int. J. Robotics Res.','Robotics: Science and Systems','Humanoids','WAFR','IROS','ICRA','FSR','ISER','ISRR','AAMAS','IEEE Robotics Autom. Lett.','IEEE Trans. Robotics and Automation'],
        'GR':['ACM Trans. Graph.','Comput. Graph. Forum','SIGGRAPH','SIGGRAPH Asia','Symposium on Computer Animation','AISTATS','IJCAI','UAI'],
             }
sTypes = ['Full'] + [k for k,v in conf_types.items()]

In [ ]:
for sub in ['ROB','CV','GR','ML']:
    print(scoreV['1/i'][:,sTypes.index(sub)].sum(1).shape)
print(scoreV['1/i'].shape)

In [ ]:
first_paper_year = 2005
last_paper_year = 2019
min_pub_years = 2
avg_auth_pos2 = avg_auth_pos[idx]/avg_auth_cnt[idx]
condition = (auth_years[:,0] >= first_paper_year).astype(np.float)
condition = (((auth_been_last < 2) )& (auth_years[:,0] >= first_paper_year) & (auth_years[:,1] >= last_paper_year) ) .astype(np.float)

cand_total = total_scores * condition* (year_span >= min_pub_years).astype(np.float) 

cand_ri = ri_scores* condition* (year_span >= min_pub_years).astype(np.float) 

cand_total_ef = cand_total/year_span
cand_ri_ef = cand_ri/year_span
cols_l = ['Rank','Author','RI','Total','eRI','eTotal','Since','Until','Last','Affiliation']
print('{}\t{:30s}\t{:s}\t{:s}\t{:s}\t{:s}\t{:s}\t{:s}\t{:s}\t{:20s}'.format(*cols_l))
export_list = []
for num,idx in enumerate(np.argsort(cand_ri)[::-1][:50000]):
    uni = faculty_lookup.get(all_authors[idx],'None')
    data_l = [num+1,all_authors[idx],
              cand_ri[idx],cand_total[idx],
              cand_ri_ef[idx],cand_total_ef[idx],
              auth_years[idx,0],auth_years[idx,1],auth_been_last[idx],uni]
    export_list.append(data_l)

    if num < 100:
        print('{}\t{:30s}\t{:.1f}\t{:.1f}\t{:.1f}\t{:.1f}\t{:.0f}\t{:.0f}\t{:.0f}\t{:20s}'.format(*data_l))



In [ ]:
pd.DataFrame(export_list,columns=cols_l).iloc[:50000].to_csv('pot_export.csv')

In [ ]:
count_of_papers = np.zeros(years_per_conf*n_confs)
for paper in all_papers:
    tag,title, authors, venue, pages, startPage,year,volume,number,url,publtype,eb_toofew,eb_skip = paper
    yr = (year-min_year)//YEAR_BLOCKS
    j = years_per_conf*conf_idx[venue] + yr
    count_of_papers[j] += 1
# safe divide
count_of_papers = np.maximum(1,count_of_papers)

In [ ]:
count_of_papers2 = count_of_papers.reshape((-1,years_per_conf))
masked_count = (count_of_papers2 * (count_of_papers2 > 1))
how_many_valid_years = (masked_count > 1).sum(1)
conf_sizes = masked_count.sum(1)/how_many_valid_years
valid_confs_for_vis = (np.isfinite(conf_sizes)) & (conf_sizes > 1)

In [ ]:
valid_confs_for_vis

In [ ]:
tmp_scores = clf_gold.reshape((-1,years_per_conf))[:,-3:].copy()
ranks = np.nan_to_num(np.true_divide(tmp_scores.sum(1),(tmp_scores!=0).sum(1)))
#ranks = clf_gold.reshape((-1,years_per_conf))[:,-3:].max(1)

In [ ]:
short_names = {'Theory of Computing, Graduate Surveys' : 'Theory of Computing, GS',
               'IEEE Symposium on Security and Privacy':'IEEE SSP',
              'IEEE Conference on Computational Complexity': 'IEEE CCC',
              'USENIX Annual Technical Conference': 'USENIX ATC',
              'Journal of Machine Learning Research': 'JMLR',
              'Computer Communication Review' : 'SIGCOMM CCR'}
things = []
for idx in np.argsort(ranks)[::-1]:#[:500]:
    name = short_names[all_venues[idx]] if all_venues[idx] in short_names else all_venues[idx]
    paper_count = conf_sizes[idx]
    if not valid_confs_for_vis[idx]:
        continue
    if ranks[idx] < 0.1:
        continue
    things.append((name,ranks[idx],conf_sizes[idx]))
    print('{:30s}\t{:.1f}\t{:.1f}'.format(name,ranks[idx],conf_sizes[idx]))


In [ ]:
pd.DataFrame(things,columns=['Name','Score','Size']).set_index('Name').to_csv('confs.csv')

In [ ]:
ranks[conf_idx['STOC']]

In [ ]:
# needs cluster_new to be run. sorry
labels = np.load('labels.npy')
mask = np.load('mask.npy')

In [ ]:
# needs cluster_new to be run. sorry
cluster_ranks = clf_gold.reshape((-1,years_per_conf))[mask,:]
small_names = np.array([all_venues[i] for i in range(n_confs) if mask[i]])
alias_dict = conf_fix#csv2dict_str_str('name_alias.csv')
group_names = csv2dict_str_str('group_names.csv')
group_names = {k:v for k,v in group_names.items() if v != ''}
valid_confs_for_small_vis = valid_confs_for_vis[mask]

In [ ]:
NUM_CLUSTERS = labels.max() + 1
small_conf_counts = np.zeros(NUM_CLUSTERS)

for i in range(NUM_CLUSTERS):
    cluster_names = small_names[labels == i]
    for conf in cluster_names:
        #clf_gold[years_per_conf*conf_idx[conf]:years_per_conf*(1+conf_idx[conf])].max() *
        small_conf_counts[i] +=  count_of_papers[years_per_conf*conf_idx[conf]:years_per_conf*(1+conf_idx[conf])].sum()

In [ ]:
import matplotlib.pyplot as plt
ROWS = int(np.ceil(NUM_CLUSTERS/4))
plt.figure(figsize=(4*ROWS,32))
names_to_save = []



for cnti, i in enumerate(np.argsort(small_conf_counts)[::-1]):
    plt.subplot(ROWS,4,cnti+1)
    cluster_names = small_names[labels == i]
    cluster_filtered = cluster_ranks[labels == i] * valid_confs_for_small_vis[labels == i].reshape((-1,1))
    
    cluster_scores = cluster_filtered[:,-10:].max(1)
    valid_idxs = np.argsort(cluster_filtered[:,-4:-1].mean(1))[::-1]
    has_name = False
    for idx in valid_idxs[:6]:
        name_for_now = cluster_names[idx]
        if not has_name and name_for_now in group_names:
            plt.title(group_names[name_for_now])
            has_name = True
            #print(group_names[name_for_now],cnti,small_conf_counts[i])
        name_for_now = alias_dict[name_for_now] if name_for_now in alias_dict else name_for_now
        line = [np.nan if _ <= 0 else _ for _ in cluster_filtered[idx]]
        s = [_ for _ in cluster_filtered[idx] if _ >=0][-1]
        plt.plot(min_year+np.arange(years_per_conf)*YEAR_BLOCKS,line,label='{:15s} ({:.1f})'.format(name_for_now[:15],s))
        names_to_save.append(cluster_names[idx])
        #print(name_for_now,i)
    plt.legend(loc=3,fancybox=True, framealpha=0.8, borderpad=1,frameon=True,prop={'stretch':0,'family': 'monospace'})
    plt.tight_layout()
    #plt.ylim(0,13)
    plt.ylim(bottom=0.05)
    plt.xlim(1970,2020)
plt.savefig('big_plot7.pdf',facecolor='w',edgecolor='w')

In [ ]:
for _ in [_ for _ in names_to_save if _ not in conf_fix and len(_) > 8]:
    print(_)